# Theory
I try out stuff from papers covering trapped BEC Theory. This is me just trying to practically "get a feel" of the various parameters involved in trapped BEC theory. 

Reference Paper:
* Dalfovo, F., Giorgini, S., Pitaevskiĭ, L.P., Pitaevskiĭ, L.P., & Stringari, S. (1998). Theory of Bose-Einstein condensation in trapped gases. Reviews of Modern Physics, 71, 463-512. https://journals.aps.org/rmp/abstract/10.1103/RevModPhys.71.463


In [ ]:
from oqtant.schemas.quantum_matter import QuantumMatterFactory
from oqtant.schemas.quantum_matter import QuantumMatter as qm
from oqtant.schemas.output import Gaussian_dist_2D
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import scipy.optimize as opt
from IPython.display import HTML

qmf = QuantumMatterFactory()
qmf.get_login()

In [ ]:
qmf.get_client()

In [ ]:
matters_it = []

# Create a quantum matter object with NO barrier for establishing a baseline BEC distribution
matter_baseline = qmf.create_quantum_matter(
    temperature=temp,
    lifetime=1,
    image="IN_TRAP",
    name="Theory #1: no barrier",
)

matters_it.append(matter_baseline)

In [ ]:
matter_tof = qmf.create_quantum_matter(
    temperature=temp,
    lifetime=1,
    image="TIME_OF_FLIGHT",
    name="Theory #1: TOF image for atom number",
)

matters_it.append(matter_tof)

In [ ]:
submission = [matter.submit(track=True) for matter in matters_it]

In [ ]:
# Get baseline data from the job without a barrier
matter_baseline.get_result()
im_baseline = matter_baseline.output.get_image_data()

In [ ]:
rows, cols = im_baseline.shape

pixcal = matter_baseline.output.it_plot.pixcal

x = np.linspace(-pixcal * cols / 2, pixcal * cols / 2, cols)
y = np.linspace(-pixcal * rows / 2, pixcal * rows / 2, rows)
xg, yg = np.meshgrid(x, y)

# Fit the experimental image to a 2D Gaussian function (imported at the beginning of the notebook)
# Order of imputs in Guassian_dist_2D (after the x-y meshgrid):
# amplitude, x_center, y_center, sigma_x, sigma_y, offset
initial_guess = [
    1,
    0,
    0,
    10,
    2,
    0,
]  # Initial 2D Gaussian parameters guess for the in-trap atom cloud size
bl = [0.3, -10, -10, 4, 0.5, -0.2]  # Lower bound for the parameters during fitting
bu = [3, 10, 10, 20, 5, 0.2]  # Upper bound for the parameters during fitting

popt, _ = opt.curve_fit(
    Gaussian_dist_2D, (xg, yg), im_baseline.ravel(), p0=initial_guess, bounds=(bl, bu)
)

# Print some of the fit parameters
sigma_x, sigma_y = (popt[3], popt[4])

print("Fit parameters: ")
print(f"Sigma-x = {sigma_x:.1f} um")
print(f"Sigma-y = {sigma_y:.1f} um")
print(f"Peak optical depth = {popt[0]:.2f}")

# Plot the experimental image
matter_baseline.output.plot_it()

In [ ]:
matter_tof.get_result()
N_atoms = matter_tof.output.condensed_atom_number
print(f"Condensed atom number: {N_atoms:.0f}")
matter_tof.output.plot_tof()

In [ ]:
# Define constants, all in SI units
hbar = 1.055e-34  # reduced Plank's constant
m = 1.443e-25  # mass of Rubidium 87
a_s = 95 * 0.529e-10  # the scattering length of rubidium-87

# Calculate the peak atom density
V = 4 / 3 * np.pi * (1e-6 * sigma_x) * (1e-6 * sigma_y) ** 2  # units of meters^3
U = 4 * np.pi * hbar**2 * a_s / m
average_density = N_atoms / V
peak_density = 2 * average_density
print(f"Peak atom number density in-trap: {1e-6*peak_density:.1E} 1/cm^3")

# Calculate the speed of sound
v_s = np.sqrt(peak_density * U / m / 2)
print(f"Speed of sound in BEC: {1e3 * v_s:.2f} mm/s")